In [ ]:
import mitsuba as mi
import drjit as dr
import numpy as np
import platform
import matplotlib.pyplot as plt

if platform.system() == 'Darwin':
    mi.set_variant('llvm_acoustic')
else:
    mi.set_variant('cuda_acoustic')

print(f'Mitsuba variant: {mi.variant()}')

In [ ]:
c = 340

spectrum = mi.load_dict({
    "type": "acoustic",
    "speed_of_sound": c,
    "frequencies": "200, 300, 500",
    "values": "0.5, 2, 1"
})
print(spectrum)

In [ ]:
si = mi.SurfaceInteraction3f()

# note that the distribution is sorted by wavelengths in ascending order
frequencies = [100, 200, 300, 400  , 500, 600]
values      = [0  , 0.5, 2  , 1.375, 1  , 0  ] # at 400 Hz the value is interpolated in wavelength space
for i in range(len(values)):
    si.wavelengths = c / frequencies[i]
    print(f'frequency: {frequencies[i]} Hz')
    print(f'    wavelength: {si.wavelengths}')
    print(f'    values[i]: {values[i]}')
    print(f'    eval(): {spectrum.eval(si)}')
    # assert dr.allclose(spectrum.eval(si), values[i])

    print(f'    PDF: {spectrum.pdf_spectrum(si)}')
    print(f'    values[i]/integral: {values[i] / 1.38833}\n') # divide by integal of pdf
    # assert dr.allclose(spectrum.pdf_spectrum(si), values[i] / 1.38833)

# Sampling of a uniform spectrum

In order to sample a uniform spectrum, we need to sample in logarithmic frequency space so that importance sampling produces equal amounts of rays in each frequency band. 

The following code shows how to sample a uniform spectrum using the `sample_spectrum()` method of the `mitsuba.Spectrum` class:

In [ ]:
c = 340

spectrum = mi.load_dict({
    "type": "acoustic",
    "speed_of_sound": c,
    "frequencies": "250, 1000",
    "values": "1, 1"
})
print(spectrum)

In [ ]:
print(f'For uniform samples s=0 and s=1, the spectrum is sampled at the min and max wavelength, respectively, which corresponds to the min and max frequency of 250 Hz and 1000 Hz.')
print('For s=0.5, the sample is at the middle of the spectrum in logarithmic frequency space (500 Hz).\n')

si = mi.SurfaceInteraction3f()

for s in [0, .5, 1]:
    # Sample the spectrum using the current uniform sample
    sample = spectrum.sample_spectrum(si, s)

    print(f'uniform sample: {s}')
    print(f'    wavelength sample: {sample[0]}')
    print(f'    frequency of sample: {c/sample[0]}')